# 데이터 전처리

In [1]:
max_length=256

In [2]:
import pandas as pd
df=pd.read_csv('sms.tsv', sep='\t', )
print(df.columns)
print(df.shape)

Index(['label', 'sms'], dtype='object')
(5572, 2)


In [3]:
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
classes= sorted(set(df['label']))
class_to_idx={}

for i, c in enumerate(classes):
    class_to_idx.update({c: i})
    
nclass=len(classes)

print("# of classes: %d" %nclass)
print(classes)
print(class_to_idx)

# of classes: 2
['ham', 'spam']
{'ham': 0, 'spam': 1}


# label, sms만 남기기

In [5]:
new_df= pd.DataFrame({'label': df['label'],
                     'sms': df['sms'].str.slice(
                     start=0, stop=max_length)
                     })

In [6]:
len(new_df)

5572

In [7]:
new_df=pd.DataFrame(new_df.drop_duplicates())

In [8]:
len(new_df)

5169

In [9]:
df_shuffled=new_df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,label,sms
0,ham,Kinda. First one gets in at twelve! Aah. Speak...
1,ham,S:-)if we have one good partnership going we w...
2,ham,"Sweet, we may or may not go to 4U to meet carl..."
3,ham,* Thought I didn't see you.
4,ham,I'm putting it on now. It should be ready for ...


In [10]:
train_ratio=0.9

s, e,=0, int(df_shuffled.shape[0]*train_ratio)
df_train=pd.DataFrame({'label': df_shuffled['label'][s:e],
                      'sms': df_shuffled['sms'][s:e]
                      })
print("index for train: %d-%d" %(s,e))

s,e=e, e+int(df_shuffled.shape[0]*(1.0-train_ratio))
print("index for test: %d-%d" %(s,e))
df_test=pd.DataFrame({'label': df_shuffled['label'][s:e],
                      'sms': df_shuffled['sms'][s:e]
                      })

index for train: 0-4652
index for test: 4652-5168


In [11]:
print(df_train.shape)
print(df_test.shape)

(4652, 2)
(516, 2)


In [12]:
df_train.to_csv('./sms.maxlen.uniq.shf.train.tsv',
               header=False, index=False, sep='\t')
df_test.to_csv('./sms.maxlen.uniq.shf.test.tsv',
               header=False, index=False, sep='\t')

# 데이터 로더

In [13]:
import torch
print(torch.__version__)

1.4.0


In [14]:
!pip install torchtext==0.4.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [15]:
import torchtext
import numpy as np

In [16]:
from data_loader import DataLoader

# RNN+SMS 구현

In [17]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

In [18]:
batch_size=128
num_epochs=10

word_vec_size=256
dropout_p=0.3

hidden_size=512
num_layers=4

learning_rate=0.001

In [19]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
loaders=DataLoader(
                    train_fn='sms.maxlen.uniq.shf.train.tsv',
                    batch_size=batch_size,
                    valid_ratio=.2,
                    device=-1,
                    max_vocab=999999,
                    min_freq=5, )

In [21]:
test_loaders=DataLoader(
                    train_fn='sms.maxlen.uniq.shf.test.tsv',
                    batch_size=batch_size,
                    valid_ratio=.01,
                    device=-1,
                    max_vocab=999999,
                    min_freq=5, )

In [22]:
print("|train| =", len(loaders.train_loader.dataset),
     "|test| =", len(loaders.valid_loader.dataset))

vocab_size=len(loaders.text.vocab)
num_classes=len(loaders.label.vocab)
print('|vocab| =', vocab_size, '|classes| =', num_classes)

|train| = 3722 |test| = 930
|vocab| = 1524 |classes| = 2


In [23]:
n=3
for i, data in enumerate(loaders.train_loader):
    labels=data.label
    texts=data.text
    
    if i>n:
        break
    print("[%d]" %i)
    print("한 번에 로드되는 데이터 크기:", len(labels))
    
    for j in range(n):
        label=labels[j].numpy()
        text=texts[j].numpy()
        print("label: ", label)
        print("text: ", text.shape)

[0]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (4,)
label:  0
text:  (4,)
label:  0
text:  (4,)
[1]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (16,)
label:  0
text:  (16,)
label:  0
text:  (16,)
[2]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (6,)
label:  0
text:  (6,)
label:  0
text:  (6,)
[3]
한 번에 로드되는 데이터 크기: 10
label:  0
text:  (60,)
label:  0
text:  (60,)
label:  0
text:  (60,)


In [24]:
class RNN(nn.Module):
    def __init__(self,
                input_size,
                word_vec_size,
                hidden_size,
                n_classes,
                num_layers=2,
                dropout_p=0.3
                ):
        super(RNN, self).__init__()
        
        self.input_size=input_size
        self.word_vec_size=word_vec_size
        self.hidden_size=hidden_size
        self.n_classes=n_classes
        self.num_layers=num_layers
        self.dropout_p=dropout_p
        
        self.emb=nn.Embedding(input_size, word_vec_size)
        
        self.lstm=nn.LSTM(input_size=word_vec_size,
                         hidden_size=hidden_size,
                         num_layers=num_layers,
                         dropout=dropout_p,
                         batch_first=True,
                         bidirectional=True)
        self.fc=nn.Linear(hidden_size*2, num_classes)
        
        self.activation=nn.LogSoftmax(dim=-1)
        
    def forward(self, x):
        x=self.emb(x)
    
        x, _=self.lstm(x)
    
        out=self.activation(self.fc(x[:,-1]))
    
        return out

In [25]:
model=RNN(input_size=vocab_size,
         word_vec_size=word_vec_size,
         hidden_size=hidden_size,
         n_classes=num_classes,
         num_layers=num_layers,
         dropout_p=dropout_p)

In [26]:
def ComputeAccr(dloader, imodel):
    correct=0
    total=0
    
    model.eval()
    for i, data in enumerate(dloader):
        texts=data.text.to(device)
        labels=data.label.to(device)
        
        output=model(texts)
        _, output_index=torch.max(output, 1)
        
        total+=labels.size(0)
        correct+=(output_index==labels).sum().float()
        
    model.train()
    return(100*correct/total).numpy()

In [27]:
print("Accruacy of Test Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accruacy of Test Data: 13.01


In [28]:
loss_func=nn.NLLLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
total_step=len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loaders.train_loader):
        texts=data.text.to(device)
        labels=data.label.to(device)
        
        print("[%d]" %i)
        
        outputs=model(texts)
        loss=loss_func(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%10==0:
            print("Epch [{}/{}]. Step[{}/{}], Loss: {:.4f}, Accr: {:.2f}"
                 .format(epoch+1, num_epochs, i+1, total_step,
                        loss.item(), ComputeAccr(loaders.valid_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epch [1/10]. Step[10/30], Loss: 0.1356, Accr: 86.99
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epch [1/10]. Step[20/30], Loss: 0.1870, Accr: 86.99
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epch [1/10]. Step[30/30], Loss: 0.0486, Accr: 86.99
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epch [2/10]. Step[10/30], Loss: 0.7753, Accr: 87.42
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epch [2/10]. Step[20/30], Loss: 0.6079, Accr: 87.42
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epch [2/10]. Step[30/30], Loss: 0.1207, Accr: 86.24
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epch [3/10]. Step[10/30], Loss: 0.5272, Accr: 94.19
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epch [3/10]. Step[20/30], Loss: 0.2392, Accr: 91.83
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epch [3/10]. Step[30/30], Loss: 0.3190, Accr: 89.03
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epch [4/10]. Step[10/30], Loss: 0.0383, Accr: 89.25
[10]
[11]
[12]
[13]


In [31]:
print("Accruacy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accruacy of Valid Data: 97.31


In [33]:
netname='../nets/rnn_weight.pkl'
torch.save(model, netname, )

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [35]:
netname='../nets/rnn_weight.pkl'
model=torch.load(netname)

In [36]:
print("Accruacy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accruacy of Valid Data: 97.31
